# Optimization Methods Project Work: SMO and DCD-Linear for training SVM

This notebook will contain some code that implements the SMO and DCD-Linear algorithm, plus the results.

## Testing the environment

The following cell will simply verify if all the libraries necessary to run the code are correctly installed.

In [1]:
import numpy as np
import sklearn
import matplotlib as plt
import math

print("Everything is good!")

Everything is good!


## Defining the Dual Problem

### The Bias One (For SMO)

As we know, the formulation of the dual problem for training a SVM is:
$$
\begin{gather*}
\min_{\alpha} \frac{1}{2}\alpha^TQ\alpha - e^T\alpha \\
\forall i\ 0 \leq \alpha_i \leq C\ \ \sum \limits_i^n \alpha_iy_i = 0
\end{gather*}
$$

As stated in the Project Goals, we will use the Most Violating Pair rule to select the variables to change. So, to implement this algorithm efficiently, we need those elements:
* A function that calculates the derivatives
* A function that adjust the derivative after changing $\alpha$ 
* A function that extract the most violating pair.

To sum up all this functionality, they will be implemented inside a class. The constructor will receive the set of Xs and Ys, and using that will derive Q.

In [57]:
class DualSVMProblem:

    def __init__(self, Xs, Ys, C, epsilon = 10e-4):
        self.X = Xs
        self.Y = Ys
        self.C = C
        self.a = np.zeros(Ys.shape[0])
        self.e = np.ones_like(self.a)
        self.epsilon = epsilon

        mat = np.stack([Ys.T] * Xs.shape[1], axis = 1)
        
        Z = Xs * mat
        self.Q = Z @ Z.T

        self.d = self.Q @ self.a - self.e

    def getA(self):
        return self.a
        
    def getDerivative(self):
        return self.d    

    def getMostViolatingPair(self):
        self.directions = self.d / self.Y
        min_idx = -1
        min_value = np.inf
        max_idx = -1
        max_value = -np.inf

        for i in range(len(self.a)):
            if (self.a[i] > self.epsilon and self.Y[i] == 1 and self.directions[i] < min_value):
                min_value = self.directions[i]
                min_idx = i
            if (self.a[i] < self.C - self.epsilon and self.Y[i] == -1 and self.directions[i] < min_value):
                min_value = self.directions[i]
                min_idx = i
                
            if (self.a[i] > self.epsilon and self.Y[i] == -1 and self.directions[i] > max_value):
                max_value = self.directions[i]
                max_idx = i
            if (self.a[i] < self.C - self.epsilon and self.Y[i] == 1 and self.directions[i] > max_value):
                max_value = self.directions[i]
                max_idx = i

        if min_idx == -1 or max_idx == -1:
            return None
                
        return (min_idx, max_idx)

    def updateA(self, idx1, a1, idx2, a2):
        self.d = self.Q[idx1] * (a1 - self.a[idx1]) + self.Q[idx2] * (a2 - self.a[idx2]) + self.d
        self.a[idx1] = a1
        self.a[idx2] = a2

    def getParameters(self):
        mat = np.stack([self.a.T * self.Y.T] * self.X.shape[1], axis = 1)
        w = np.sum(mat, axis = 0)
        b = 0
        for i in range(len(self.a)):
            if(self.a < self.epsilon or self.a > self.C - self.epsilon):
                continue
            b = 1 / self.Y[0] - self.w @ self.X[0]
            break        
        return (w, b)

class SVM:
    def __init__(self, w, b):
        self.w = w
        self.b = b

    def __call__(self, x):
        return np.sign(x @ w + b)

In [58]:
# Let's test the code with some artificial data
X = np.array([[1, 2, 3], [4, 5, 6]])
Y = np.array([1, -1])
C = 5

# The safe way to calculate matrix Q
Q = np.zeros((2, 2))
for i in range(2):
    for j in range(2):
        Q[i][j] = Y[i] * Y[j] * X[i] @ X[j]

problem = DualSVMProblem(X, Y, C)

ok = True
if ((Q != problem.Q).all()):
    print("The matix calculation is wrong")
    
if ((np.array([-1, -1]) != problem.getDerivative()).all()):
    print("The derivative function if wrong")

update = np.array([1, 2])
problem.updateA(0, update[0], 1, update[1])

if ((Q @ update - np.ones(2) != problem.getDerivative()).all()):
    print("The updates of variables doesn't update the derivative in the right way")

elements = problem.getDerivative() / Y

if not (problem.getMostViolatingPair()[0] == (1 if elements[0] > elements[1] else 0) and problem.getMostViolatingPair()[1] == (1 if elements[1] > elements[0] else 0)):
    print("The calculation of the most violating pair is wrong")
    # This test is a bit odd and not exaustive, but can help

if ok:
    print("Everything is working as expected!")

Everything is working as expected!


## Implementation of SMO

After implementing core elements for the SMO algorithm, it's time to get them together and build the actual result.
In this implementation, I realized a function that execute a single iteration of the algorithm, leaving the check of convergence outside. The objective is to reuse the external function with the different approach (DCD-Linear), and then make easier to instrument the code to measure some components like loss, validation, etc.

In [59]:
from typing import Callable
def training(problem: DualSVMProblem, step: Callable[[DualSVMProblem], bool], loss_func: Callable[[SVM], float] = None, validation_func: Callable[[SVM], float] = None, epsilon = 10e-5):
    loss = []
    calidation = []
    
    while(True): 
        (min_idx, max_idx) = problem.getMostViolatingPair()
        elements = problem.getDerivative() / Y
        
        if(elements[min_idx] + epsilon > elements[max_idx]):
            break
        
        result = step(problem)            
        (w, b) = problem.getParameters()
        svm = SVM(w, b)
        if(loss_func is not None):
            loss.append(loss_func())
    
        if(validation_func is not None):
            validation.append(validation_func())

        if not result:
            # If the step method give False as a result indicates an error or a stop condition (i.e. aving a derivative under the tollerance)
            break

    return (loss, validation)

def SMO_step(problem: DualSVMProblem, epsilon: float = 10e-5):
    test = problem.getMostViolatingPair()
    if(test is None):
        raise Exception("The problem did not give a valid violating pair")
    (min_idx, max_idx) = (test[0], test[1])

    direction = np.zeros(len(problem.a))
    direction[min_idx] = 1 * problem.Y[min_idx]
    direction[max_idx] = - 1 * problem.Y[max_idx] #In theory I should divide, but y is in {-1, 1} making the multiplication equivalent, but more efficient

    derivative = problem.getDerivative()

    b = problem.C - problem.a[min_idx] if direction[min_idx] > 0 else problem.a[min_idx]
    b = min(b, problem.C - problem.a[max_idx] if direction[max_idx] > 0 else problem.a[max_idx])

    if b < epsilon:
        return False

    value = direction.T @ problem.Q @ direction

    if value > epsilon:
        b = min(b, -(derivative.T @ direction) / value)

    problem.updateA(0, problem.a[min_idx] + b * direction[min_idx], 1, problem.a[max_idx] + b * direction[max_idx])

    return True    

## Testing the Code

In the following cells, I will use a very simple classification problem to see if the code works at all. [This dataset](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html#breast-cancer) is composed by only 683 elements, everyone of only 10 features. This dataset is chosen dues to it contained dimension. In fact, this implementation of the Dual SVM explicitly calculate the Q matrix, something that can become prohibitive very quickly in the increase of elements.

In [94]:
from sklearn.datasets import load_svmlight_file
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

(Xs, Ys) = load_svmlight_file("breast_cancer_scale")
Ys[Ys == 2] = 1
Ys[Ys == 4] = -1 # TODO control the description to decide what is positive and what is negative

Xs = Xs.toarray()

Xs 

# There is a lot to say (Xs originally is a sparce matrix, not scaled values broke the model, Ys classes are 2 and 4)

array([[-0.860107, -0.111111, -1.      , ..., -0.555556, -1.      ,
        -1.      ],
       [-0.859671, -0.111111, -0.333333, ..., -0.555556, -0.777778,
        -1.      ],
       [-0.857807, -0.555556, -1.      , ..., -0.555556, -1.      ,
        -1.      ],
       ...,
       [-0.876716, -0.111111,  1.      , ...,  0.555556,  1.      ,
        -0.777778],
       [-0.875424, -0.333333,  0.555556, ...,  1.      ,  0.111111,
        -1.      ],
       [-0.875424, -0.333333,  0.555556, ...,  1.      , -0.333333,
        -1.      ]], shape=(683, 10))

In [95]:
(train_Xs, test_Xs, train_Ys, test_Ys)  = train_test_split(Xs, Ys, test_size = 0.20)

base_svm = LinearSVC(C = 10)
base_svm.fit(train_Xs, train_Ys)

train_preds = base_svm.predict(train_Xs)
test_preds = base_svm.predict(test_Xs)


print("Report on train set:")
print(classification_report(train_Ys, train_preds, zero_division = 0))

print("Report on test set:")
print(classification_report(test_Ys, test_preds, zero_division = 0))

Report on train set:
              precision    recall  f1-score   support

        -1.0       0.95      0.96      0.95       188
         1.0       0.98      0.97      0.98       358

    accuracy                           0.97       546
   macro avg       0.96      0.97      0.97       546
weighted avg       0.97      0.97      0.97       546

Report on test set:
              precision    recall  f1-score   support

        -1.0       1.00      0.92      0.96        51
         1.0       0.96      1.00      0.98        86

    accuracy                           0.97       137
   macro avg       0.98      0.96      0.97       137
weighted avg       0.97      0.97      0.97       137



In [105]:
base_problem = DualSVMProblem(train_Xs, train_Ys, C = 1)

print(base_problem.getMostViolatingPair())
d = base_problem.getDerivative()
t = d / train_Ys
print(t[5], t[0])


SMO_step(base_problem)

(5, 0)
1.0 -1.0
0.0


False